In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#!pip install arch
from arch import arch_model

# Load the data

prices = pd.read_csv('europe indices monthly.csv', delimiter=';', index_col='Date', parse_dates=True)
# Convert the columns with stock prices to numeric
prices['IBEX35'] = pd.to_numeric(prices['IBEX35'].str.replace(',', ''), errors='coerce')
prices['IT40'] = pd.to_numeric(prices['IT40'].str.replace(',', ''), errors='coerce')
prices['EUSTX50'] = pd.to_numeric(prices['EUSTX50'].str.replace(',', ''), errors='coerce')
prices['CAC40'] = pd.to_numeric(prices['CAC40'].str.replace(',', ''), errors='coerce')
prices['DAX'] = pd.to_numeric(prices['DAX'].str.replace(',', ''), errors='coerce')
print(prices)
returns = prices.pct_change().dropna()
print(returns)

             IBEX35      IT40  EUSTX50    CAC40      DAX
Date                                                    
2010-01-01  10947.7  22050.45  2953.63  3739.46   5607.5
2010-02-01  10333.6  22466.57  3013.09  3708.80   5594.5
2010-03-01  10871.3  21727.44  2910.91  3974.01   6141.5
2010-04-01  10492.2  22417.96  3011.25  3816.99   6130.5
2010-05-01   9359.4  21109.75  2861.92  3507.56   5962.5
...             ...       ...      ...      ...      ...
2021-08-01   8846.6  21559.32  3517.25  6680.18  15800.0
2021-09-01   8796.3  20648.85  3318.20  6520.01  15288.0
2021-10-01   9057.7  22652.11  3617.54  6830.34  15664.0
2021-11-01   8305.1  24610.29  3964.72  6721.16  15143.0
2021-12-01   8713.8  23706.96  3793.62  7153.03  15856.0

[144 rows x 5 columns]
              IBEX35      IT40   EUSTX50     CAC40       DAX
Date                                                        
2010-02-01 -0.056094  0.018871  0.020131 -0.008199 -0.002318
2010-03-01  0.052034 -0.032899 -0.033912  0.071508  

In [5]:
from statsmodels.tsa.stattools import coint, adfuller
import itertools
# Assuming 'returns_dict' contains the returns data for your 5 indices
returns_dict = {
    'DAX': returns['DAX'],
    'IT40': returns['IT40'],
    'IBEX35': returns['IBEX35'],
    'EUSTX50': returns['EUSTX50'],
    'CAC40': returns['CAC40'],
}

# Create a list of all indices
indices = list(returns_dict.keys())

# Perform Engle-Granger Two-Step Method for all pairs of indices
for pair in itertools.combinations(indices, 2):
    index_1 = pair[0]
    index_2 = pair[1]
    
    # Extract the returns series for the selected pair
    returns_1 = returns_dict[index_1]
    returns_2 = returns_dict[index_2]

    # Perform the Engle-Granger Two-Step Method
    cointegration_test = coint(returns_1, returns_2)
    print(f"Cointegration between {index_1} and {index_2}:")
    print("Cointegration t-statistic:", cointegration_test[0])
    print("Critical value at 5% significance:", cointegration_test[2][1])
    
    # Check if the pair is cointegrated
    if cointegration_test[0] < cointegration_test[2][1]:
        print(f"There is evidence of cointegration between {index_1} and {index_2}.")
        
        # Step 2: Calculate residuals and apply ADF test
        residuals = returns_1 - cointegration_test[0] * returns_2
        adf_test_result = adfuller(residuals)
        print("ADF Test p-value for residuals:", adf_test_result[1])
        if adf_test_result[1] < 0.05:  # You can choose a significance level
            print("Residuals are stationary, suggesting cointegration.")
        else:
            print("Residuals are not stationary, suggesting no cointegration.")
    else:
        print(f"There is no evidence of cointegration between {index_1} and {index_2}.")

Cointegration between DAX and IT40:
Cointegration t-statistic: -10.12037837287681
Critical value at 5% significance: -3.379497248561793
There is evidence of cointegration between DAX and IT40.
ADF Test p-value for residuals: 1.434374808556586e-23
Residuals are stationary, suggesting cointegration.
Cointegration between DAX and IBEX35:
Cointegration t-statistic: -6.155562465559925
Critical value at 5% significance: -3.379497248561793
There is evidence of cointegration between DAX and IBEX35.
ADF Test p-value for residuals: 1.0740957657869315e-23
Residuals are stationary, suggesting cointegration.
Cointegration between DAX and EUSTX50:
Cointegration t-statistic: -10.17772499856697
Critical value at 5% significance: -3.379497248561793
There is evidence of cointegration between DAX and EUSTX50.
ADF Test p-value for residuals: 2.978816056646491e-23
Residuals are stationary, suggesting cointegration.
Cointegration between DAX and CAC40:
Cointegration t-statistic: -9.91028277632042
Critical v

In [15]:
# Perform the ADF test for each series
adf_results = {}
for column in returns.columns:
    adf_result = adfuller(returns[column])
    adf_results[column] = {
        'ADF Statistic': adf_result[0],
        'p-value': adf_result[1],
        'Critical Values': adf_result[4]
    }

# Print ADF test results
for column, result in adf_results.items():
    print(f"ADF Test for {column}:")
    print(f"ADF Statistic: {result['ADF Statistic']}")
    print(f"p-value: {result['p-value']}")
    print("Critical Values:")
    for key, value in result['Critical Values'].items():
        print(f"   {key}: {value}")

# Check for cointegration using Johansen's test
def perform_johansen_test(data):
    result = coint_johansen(data, det_order=0, k_ar_diff=1)
    print("\nJohansen Cointegration Test Results:")
    print("Eigenvalues:")
    print(result.eig)
    print("Critical Values (Trace Test):")
    print(result.cvt)
    print("Maximum Eigenvalue Statistic:")
    print(result.lr2)
    print("Critical Values (Max Eigenvalue Test):")
    print(result.cvm)
    print("Cointegration Rank (Number of Cointegrating Relations):")
    print(result.cvm)

# Check for cointegration using Johansen's test on the returns
perform_johansen_test(returns)

ADF Test for IBEX35:
ADF Statistic: -12.78224593179981
p-value: 7.372962365661328e-24
Critical Values:
   1%: -3.477261624048995
   5%: -2.8821181874544233
   10%: -2.5777431104939494
ADF Test for IT40:
ADF Statistic: -12.456876785094549
p-value: 3.475079147815655e-23
Critical Values:
   1%: -3.477261624048995
   5%: -2.8821181874544233
   10%: -2.5777431104939494
ADF Test for EUSTX50:
ADF Statistic: -12.269158727596404
p-value: 8.720299329634248e-23
Critical Values:
   1%: -3.477261624048995
   5%: -2.8821181874544233
   10%: -2.5777431104939494
ADF Test for CAC40:
ADF Statistic: -12.047371371767863
p-value: 2.6457176967716855e-22
Critical Values:
   1%: -3.477261624048995
   5%: -2.8821181874544233
   10%: -2.5777431104939494
ADF Test for DAX:
ADF Statistic: -9.979755115187004
p-value: 2.130168898762456e-17
Critical Values:
   1%: -3.4776006742422374
   5%: -2.882265832283648
   10%: -2.5778219289774156

Johansen Cointegration Test Results:
Eigenvalues:
[0.48843806 0.43897755 0.36218

In [21]:
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.vector_ar.vecm import coint_johansen
# Perform the ADF test for each series

adf_results = {}
for column in returns.columns:
    adf_result = adfuller(returns[column])
    adf_results[column] = {
        'ADF Statistic': adf_result[0],
        'p-value': adf_result[1],
        'Critical Values': adf_result[4]
    }

# Print ADF test results
for column, result in adf_results.items():
    print(f"ADF Test for {column}:")
    print(f"ADF Statistic: {result['ADF Statistic']}")
    print(f"p-value: {result['p-value']}")
    print("Critical Values:")
    for key, value in result['Critical Values'].items():
        print(f"   {key}: {value}")

# Check for cointegration using Johansen's test
def perform_johansen_test(data):
    result = coint_johansen(data, det_order=0, k_ar_diff=1)
    print("\nJohansen Cointegration Test Results:")
    print("Eigenvalues:")
    print(result.eig)
    print("Critical Values (Trace Test):")
    print(result.cvt)
    print("Maximum Eigenvalue Statistic:")
    print(result.lr2)
    print("Critical Values (Max Eigenvalue Test):")
    print(result.cvm)
    
    # Manually interpret cointegration rank based on the Maximum Eigenvalue Statistic
    eigenvalues = result.eig
    max_eigenvalue_statistic = result.lr2
    critical_values_max_eigenvalue = result.cvm
    
    # You can set your own threshold for significance (e.g., 0.05)
    significance_level = 0.05
    
    cointegration_rank = sum(max_eigenvalue_statistic > critical_values_max_eigenvalue[:, 1])
    print(f"Cointegration Rank (Number of Cointegrating Relations): {cointegration_rank}")

# Check for cointegration using Johansen's test on the returns
perform_johansen_test(returns)

ADF Test for IBEX35:
ADF Statistic: -12.78224593179981
p-value: 7.372962365661328e-24
Critical Values:
   1%: -3.477261624048995
   5%: -2.8821181874544233
   10%: -2.5777431104939494
ADF Test for IT40:
ADF Statistic: -12.456876785094549
p-value: 3.475079147815655e-23
Critical Values:
   1%: -3.477261624048995
   5%: -2.8821181874544233
   10%: -2.5777431104939494
ADF Test for EUSTX50:
ADF Statistic: -12.269158727596404
p-value: 8.720299329634248e-23
Critical Values:
   1%: -3.477261624048995
   5%: -2.8821181874544233
   10%: -2.5777431104939494
ADF Test for CAC40:
ADF Statistic: -12.047371371767863
p-value: 2.6457176967716855e-22
Critical Values:
   1%: -3.477261624048995
   5%: -2.8821181874544233
   10%: -2.5777431104939494
ADF Test for DAX:
ADF Statistic: -9.979755115187004
p-value: 2.130168898762456e-17
Critical Values:
   1%: -3.4776006742422374
   5%: -2.882265832283648
   10%: -2.5778219289774156

Johansen Cointegration Test Results:
Eigenvalues:
[0.48843806 0.43897755 0.36218

In [22]:
# Apply Johansen Cointegration Test for multiple lag orders
lag_orders = range(1, 6)  # Try lag orders from 1 to 5
best_lag_order = None
best_aic = float('inf')

for lag_order in lag_orders:
    result = coint_johansen(returns, det_order=0, k_ar_diff=lag_order)
    
    # Calculate AIC for this lag order
    aic = -2 * result.lr1[1] + 2 * np.log(len(returns))
    
    if aic < best_aic:
        best_aic = aic
        best_lag_order = lag_order

# Perform the Johansen Cointegration Test with the chosen lag order
result = coint_johansen(returns, det_order=0, k_ar_diff=best_lag_order)

# Print the results for the chosen lag order
print(f"Optimal Lag Order (Based on AIC): {best_lag_order}")
print("\nJohansen Cointegration Test Results:")
print("Eigenvalues:")
print(result.eig)
# Print other test results as needed

Optimal Lag Order (Based on AIC): 1

Johansen Cointegration Test Results:
Eigenvalues:
[0.48843806 0.43897755 0.36218757 0.35696634 0.34434873]
